In [2]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import random

# --- 1. Konfigurasi Global dan Terjemahan ---

# Pilihan inti tetap dalam bahasa Inggris untuk logika
CHOICES = ['rock', 'scissors', 'paper']
CURRENT_LANGUAGE = 'id' # Default bahasa Indonesia

# Kamus terjemahan untuk semua teks dalam game
translations = {
    'id': {
        'title': "Batu, Gunting, Kertas",
        'choose_lang': "Pilih Bahasa:",
        'lang_id': "🇮🇩 Bahasa Indonesia",
        'lang_en': "🇬🇧 English",
        'choice_label': "Pilihanmu:",
        'result_label': "Hasil Pertandingan:",
        'arena_label': "Arena Pertandingan:",
        'player_label': "KAMU",
        'comp_label': "KOMPUTER",
        'vs': "VS",
        'win': "MENANG",
        'lose': "KALAH",
        'draw': "SERI",
        'rock': "Batu",
        'scissors': "Gunting",
        'paper': "Kertas",
        'choose_prompt': "Pilih!",
        'comp_prompt': "PIL. KOMP."
    },
    'en': {
        'title': "Rock, Paper, Scissors",
        'choose_lang': "Choose Language:",
        'lang_id': "🇮🇩 Bahasa Indonesia",
        'lang_en': "🇬🇧 English",
        'choice_label': "Your Choice:",
        'result_label': "Match Result:",
        'arena_label': "Battle Arena:",
        'player_label': "YOU",
        'comp_label': "COMPUTER",
        'vs': "VS",
        'win': "WIN",
        'lose': "LOSE",
        'draw': "DRAW",
        'rock': "Rock",
        'scissors': "Scissors",
        'paper': "Paper",
        'choose_prompt': "Choose!",
        'comp_prompt': "COMP. CHOICE"
    }
}

# Mapping Emoji
EMOJI_MAP = {
    'rock': '👊',
    'scissors': '✂️',
    'paper': '🖐️',
    'default': '❓'
}

# Variabel Layout
BOX_HEIGHT = '100px'
BOX_WIDTH = '120px'

# Kontainer (perlu diinisialisasi sebelum digunakan)
output_result_area = widgets.Output(layout=widgets.Layout(width='150px', align_items='center'))
user_symbol_output = widgets.Output()
computer_symbol_output = widgets.Output()
user_choices_box = widgets.HBox([]) # Placeholder untuk tombol
main_game_display = widgets.VBox([]) # Placeholder untuk tampilan utama

# --- 2. Fungsi Utama Game ---

def get_text(key):
    """Fungsi pembantu untuk mengambil terjemahan."""
    return translations[CURRENT_LANGUAGE].get(key, key)

def get_computer_choice():
    """Memilih pilihan komputer secara acak."""
    return random.choice(CHOICES)

def build_game_ui():
    """Membangun antarmuka game (tombol, label, dan tata letak)."""
    global user_choices_box, main_game_display

    # --- Hapus tampilan lama ---
    if hasattr(main_game_display, 'close'):
        main_game_display.close()

    # --- 2.1: Buat Tombol Pilihan User (dengan teks bahasa yang benar) ---
    buttons_list = []
    styles_map = {'rock': 'info', 'scissors': 'warning', 'paper': 'success'}

    for choice_en in CHOICES:
        # Teks tombol adalah Emoji + Terjemahan
        button_label = f"{EMOJI_MAP[choice_en]} {get_text(choice_en)}"

        button = widgets.Button(
            description=button_label,
            button_style=styles_map[choice_en],
            layout=widgets.Layout(width=BOX_WIDTH, height=BOX_HEIGHT)
        )
        button.on_click(handle_click)
        buttons_list.append(button)

    user_choices_box = widgets.HBox(buttons_list)

    # --- 2.2: Bagian Bawah: Tampilan Simbol Pertandingan ---
    vs_label = widgets.Label(
        value=get_text('vs'),
        style={'font_weight': 'bold', 'font_size': '30px'},
        layout=widgets.Layout(margin='0 10px', align_self='center')
    )

    bottom_section = widgets.VBox([
        widgets.Label(get_text('arena_label'), style={'font_weight': 'bold', 'text_align': 'center', 'font_size': '18px'}),
        widgets.HBox([
            widgets.VBox([
                widgets.Label(get_text('player_label')),
                user_symbol_output
            ], layout=widgets.Layout(align_items='center')),
            vs_label,
            widgets.VBox([
                widgets.Label(get_text('comp_label')),
                computer_symbol_output
            ], layout=widgets.Layout(align_items='center')),
        ], layout=widgets.Layout(justify_content='center', margin='10px 0'))
    ])

    # --- 2.3: Bagian Atas: Pilihan User dan Hasil ---
    user_controls_stack = widgets.VBox([
        widgets.Label(get_text('choice_label'), style={'font_weight': 'bold'}),
        user_choices_box,
        widgets.Label(get_text('result_label'), style={'font_weight': 'bold', 'text_align': 'center', 'margin_top': '10px'}),
        output_result_area
    ], layout=widgets.Layout(align_items='center'))

    # --- 2.4: Gabungkan semua bagian ---
    main_game_display = widgets.VBox([
        widgets.HBox([user_controls_stack], layout=widgets.Layout(justify_content='center')),
        widgets.HBox([], layout=widgets.Layout(height='2px', background_color='#ccc', width='100%', margin='15px 0')),
        bottom_section
    ])

    # Tampilkan UI baru
    display(main_game_display)

    # Inisialisasi tampilan simbol di bawah
    update_symbol_display(user_symbol_output, None, is_user=True)
    update_symbol_display(computer_symbol_output, None, is_user=False)

# --- Fungsi Update Tampilan dan Logika ---

def update_symbol_display(output_widget, choice_en, is_user=False):
    """Memperbarui tampilan simbol pemain atau komputer."""
    border_color = '#28a745' if is_user else '#dc3545'
    style = f"border: 3px solid {border_color}; padding: 10px; font-weight: bold; background-color: #f0f0f0; border-radius: 5px; text-align: center; color: #333; height: {BOX_HEIGHT}; width: {BOX_WIDTH}; display: flex; flex-direction: column; justify-content: center; align-items: center;"

    with output_widget:
        clear_output(wait=True)

        emoji = EMOJI_MAP.get(choice_en, EMOJI_MAP['default'])

        # Teks di bawah simbol menggunakan terjemahan
        if choice_en:
            choice_text = get_text(choice_en).upper()
        else:
            choice_text = get_text('choose_prompt').upper() if is_user else get_text('comp_prompt').upper()

        display(widgets.HTML(f"""
            <div style='{style}'>
                <span style='font-size: 48px;'>{emoji}</span>
                <span style='font-size: 16px; margin-top: 5px;'>{choice_text}</span>
            </div>
        """))


def determine_winner(user_choice_en, computer_choice_en):
    """Menentukan pemenang dan memperbarui semua hasil."""

    # 1. Update Tampilan Simbol Berdampingan
    update_symbol_display(user_symbol_output, user_choice_en, is_user=True)
    update_symbol_display(computer_symbol_output, computer_choice_en, is_user=False)

    # 2. Tentukan Hasil (Logika menggunakan EN)
    if user_choice_en == computer_choice_en:
        result_key = 'draw'
        color = "#ffc107"
    elif (user_choice_en == 'rock' and computer_choice_en == 'scissors') or \
         (user_choice_en == 'scissors' and computer_choice_en == 'paper') or \
         (user_choice_en == 'paper' and computer_choice_en == 'rock'):
        result_key = 'win'
        color = "#28a745"
    else:
        result_key = 'lose'
        color = "#dc3545"

    # 3. Tampilkan Hasil Besar di Output Area (menggunakan terjemahan)
    with output_result_area:
        clear_output(wait=True)
        html_style = f"text-align: center; font-size: 32px; font-weight: bold; margin: 0; padding: 0; color: {color}; overflow: hidden;"
        result_text = get_text(result_key)
        display(widgets.HTML(f"<div style='{html_style}'>{result_text.upper()}!</div>"))


def handle_click(b):
    """Fungsi yang dipanggil saat salah satu tombol diklik."""
    # Ambil pilihan dalam format kunci bahasa Inggris ('rock', 'scissors', 'paper')
    # b.description = '👊 Batu' atau '👊 Rock'
    # Ambil kata kedua, cari kuncinya dari kamus terbalik

    text_choice = b.description.split(' ')[1].lower()

    # Cari kunci EN berdasarkan teks yang diklik
    user_choice_en = next(
        (key for key, value in translations[CURRENT_LANGUAGE].items()
         if value.lower() == text_choice), None
    )

    if user_choice_en:
        computer_choice_en = get_computer_choice()
        determine_winner(user_choice_en, computer_choice_en)

# --- 3. Pilihan Bahasa dan Inisialisasi ---

lang_output = widgets.Output()

def set_language(b):
    """Mengatur bahasa dan membangun ulang UI game."""
    global CURRENT_LANGUAGE

    if b.description == translations['id']['lang_id']:
        CURRENT_LANGUAGE = 'id'
    elif b.description == translations['en']['lang_en']:
        CURRENT_LANGUAGE = 'en'

    with lang_output:
        clear_output(wait=True)
        print(f"Bahasa diatur ke: {translations[CURRENT_LANGUAGE]['title']}")

    build_game_ui() # Bangun UI game dalam bahasa baru

# Tombol Pilihan Bahasa
id_button = widgets.Button(description=translations['id']['lang_id'], button_style='primary')
en_button = widgets.Button(description=translations['en']['lang_en'], button_style='primary')

id_button.on_click(set_language)
en_button.on_click(set_language)

# Tampilan Pilihan Bahasa
lang_selector = widgets.VBox([
    widgets.Label(translations['en']['choose_lang']),
    widgets.HBox([id_button, en_button])
])

print(f"--- {translations['en']['title']} ---")
display(lang_selector, lang_output)


--- Rock, Paper, Scissors ---


Output()